In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

In [ ]:
guaguabao = pd.read_excel(r'C:\Users\User\VSCode\CDSS\source\Clean\呱呱包.xlsx', dtype={'病歷號碼':'str'})

In [ ]:
guaguabao.columns

# guaguabao[['病歷號碼', '住院號']].drop_duplicates().to_excel(r"C:\Users\User\VSCode\CDSS\output\醫療費用清單_0531.xlsx", index=False)

In [ ]:
guaguabao['age'] = ((guaguabao.住院日 - pd.to_datetime((guaguabao['生日(年月日)']+19110000).astype('str')))/np.timedelta64(1,'Y')).astype('int')
guaguabao['AGE'] = guaguabao.groupby('病歷號碼')['age'].transform('max')
guaguabao['Over_18'] = np.where(guaguabao['AGE']>18, 1, 0)

In [ ]:
aabbcc = guaguabao[['病歷號碼', '住院號']].drop_duplicates().sort_values(['病歷號碼', '住院號'])

aabbcc['A_Count'] = aabbcc.groupby(['病歷號碼'])['住院號'].rank(method="first", ascending=True)

aabbcc['Patient_Code'] = aabbcc.病歷號碼.astype('str') + '_' + aabbcc.A_Count.astype('int').astype('str')

aabbcc = aabbcc[['病歷號碼', '住院號', 'Patient_Code']]

In [ ]:
Jim = guaguabao[['病歷號碼', '住院號', '性別', 'AGE', 'Over_18', '採檢日', '菌名', 'ICU', 'Death', '死亡日期','住院日', '出院日', '住院天數']].drop_duplicates()

Jim_condition = [
    (Jim.菌名.isin(['Staphylococcus aureus(MRSA)', 'Staphylococcus aureus'])),
    (Jim.菌名.isin(['Klebsiella pneumoniae'])),
    (Jim.菌名.isin(['Enterococcus faecium(VRE)', 'Enterococcus faecium'])),
    (Jim.菌名.isin(['Acinetobacter baumannii', 'Acinetobacter baumannii/calcoaceticus complex'])),
    (Jim.菌名.isin(['Pseudomonas aeruginosa'])),
    (Jim.菌名.isin(['Escherichia coli'])),
    (Jim.菌名.isin(['Enterococcus faecalis', 'Enterococcus faecalis(VRE)'])),
    (Jim.菌名.isin(['Morganella morganii']))
]

Jim_value = ['SA', 'KP', 'E.faecium', 'AB', 'PA', 'ECL', 'E.faecalis', 'MM'
    
]

Jim['Group_Of_Bacteria'] = np.select(Jim_condition, Jim_value)
# Jim['Count_Of_Bacteria'] = Jim.groupby('病歷號碼')['Group_Of_Bacteria'].transform('count')

Jim['ICU_sum'] = Jim.groupby('病歷號碼')['ICU'].transform('sum')
Jim['ICU'] = np.where(Jim['ICU_sum']>=1, 1, 0)

# Jim = Jim[['病歷號碼', '住院號', '採檢日', 'Group_Of_Bacteria']].drop_duplicates().sort_values(['病歷號碼', '採檢日']).reset_index(drop=True)

In [ ]:
# Jim['Exam_min'] = Jim.groupby('病歷號碼')['採檢日'].transform('min')

TableJim = pd.merge(Jim, aabbcc, how='left', on=['病歷號碼', '住院號'])

In [ ]:
# JIm_min = Jim[Jim.Exam_min==Jim.採檢日][['病歷號碼', '住院號', '採檢日']].drop_duplicates()

In [ ]:
TableJim.columns

In [ ]:
Day = TableJim[['病歷號碼', '住院號', '住院日', '出院日', '住院天數']].drop_duplicates().sort_values(['病歷號碼', '住院日']).reset_index(drop=True)
Day['Length_of_Stay'] = Day.groupby('病歷號碼')['住院天數'].transform('sum')
Day['住院日_next'] = Day.groupby('病歷號碼')['住院日'].shift(-1)
Day['住院_Diff'] = (Day['住院日_next'] - Day['出院日']).dt.days
Day['Over30_Stay_All'] = np.where(Day.住院_Diff >= 30, 1, 0)

ExamDay = TableJim[['病歷號碼', '住院號', '住院日', '採檢日', '出院日']].drop_duplicates().sort_values(['病歷號碼', '住院號', '採檢日']).reset_index(drop=True)
ExamDay['採檢日_Earliest'] = ExamDay.groupby(['病歷號碼', '住院號'])['採檢日'].transform('min')
# ExamDay = ExamDay[ExamDay.採檢日==ExamDay.採檢日_Earliest]
# ExamDay['住院天數_Exam'] = (ExamDay.出院日 - ExamDay.採檢日).dt.days
# ExamDay['Length_of_Stay_Exam'] = ExamDay.groupby('病歷號碼')['住院天數_Exam'].transform('sum')

# drop whithout length of stay # 7 people
# ExamDay = ExamDay[~(ExamDay.住院天數_Exam.isna())]

## MRSA Exam
# ExamDay_MRSA = ExamDay[ExamDay.Bacterial_Positive==1]
# ExamDay_MRSA['採檢日_Next'] = ExamDay_MRSA.groupby(['病歷號碼', '出院日'])['採檢日'].shift(-1)
# ExamDay_MRSA['採檢日_diff'] = (ExamDay_MRSA['採檢日_Next']-ExamDay_MRSA['採檢日']).dt.days
# ExamDay_MRSA['Over30_Exam'] = np.where(ExamDay_MRSA['採檢日_diff']>=30, 1, 0)

## MRSA 住離院
StayDay = TableJim[['病歷號碼', '住院號', '住院日', '出院日']].drop_duplicates().sort_values(['病歷號碼', '住院號', '住院日'])
StayDay['住院日_Next'] = StayDay.groupby(['病歷號碼'])['住院日'].shift(-1)
StayDay['住院日_diff'] = (StayDay['住院日_Next']-StayDay['出院日']).dt.days
StayDay['Over30_Stay'] = np.where(StayDay['住院日_diff']>=30, 1, 0)

In [ ]:
import msoffcrypto
import io

temp_1 = io.BytesIO()
temp_2 = io.BytesIO()

In [ ]:
with open(r'C:\Users\User\VSCode\CDSS\source\智抗菌住院病人費用_2021名單.xlsx', 'rb') as f:
    excel = msoffcrypto.OfficeFile(f)
    excel.load_key('13226')
    excel.decrypt(temp_1)
    
Fee_1 = pd.read_excel(temp_1, dtype={'病歷號':'str'}, sheet_name='醫療費用')
del temp_1


with open(r'C:\Users\User\VSCode\CDSS\source\智抗菌住院病人費用_1110523追加.xlsx', 'rb') as f:
    excel = msoffcrypto.OfficeFile(f)
    excel.load_key('13226')
    excel.decrypt(temp_2)
    
Fee_2 = pd.read_excel(temp_2, dtype={'病歷號':'str'})
del temp_2

Fee = pd.concat([Fee_1, Fee_2]).sort_values(['住院號']).reset_index(drop=True)

In [ ]:
# ExamDay_MRSA[ExamDay_MRSA['Over30_Exam']==1].病歷號碼.to_excel(r'C:\Users\User\VSCode\CDSS\output\Exam_Over30.xlsx', index=False)

Pneumonia = pd.merge(StayDay, aabbcc, how='left', on=['病歷號碼', '住院號'])
Pneumonia = pd.merge(Pneumonia, Fee, how='left', on=['住院號'])

Pneumonia_1 = Pneumonia[Pneumonia['Over30_Stay']==1].drop_duplicates()
Pneumonia_1['Length of Stay'] = (Pneumonia_1.出院日 - Pneumonia_1.住院日).dt.days

Pneumonia_2 = Pneumonia[~(Pneumonia['Over30_Stay']==1)].drop_duplicates()
Pneumonia_2['Days of Stay'] = (Pneumonia_2.出院日 - Pneumonia_2.住院日).dt.days
Pneumonia_2['醫療總費用'] = Pneumonia_2.groupby('病歷號碼')['醫療總費用'].transform('sum')
Pneumonia_2['西藥費'] = Pneumonia_2.groupby('病歷號碼')['西藥費'].transform('sum')
Pneumonia_2['Length of Stay'] = Pneumonia_2.groupby('病歷號碼')['Days of Stay'].transform('sum')
Pneumonia_2['住院日_Mini'] = Pneumonia_2.groupby('病歷號碼')['住院日'].transform('min')
Pneumonia_2 = Pneumonia_2[Pneumonia_2['住院日_Mini']==Pneumonia_2['住院日']][['病歷號碼', '住院號', '住院日', '出院日', '住院日_Next', '住院日_diff', 'Over30_Stay', 'Patient_Code', 'Length of Stay', '醫療總費用', '西藥費']]

Pneumonia = pd.concat([Pneumonia_1, Pneumonia_2])
Pneumonia = Pneumonia[['Patient_Code', '病歷號碼', '住院號', '住院日', '出院日', '住院日_diff', 'Over30_Stay', 'Length of Stay', '醫療總費用', '西藥費']].sort_values(['病歷號碼', '住院號']).reset_index(drop=True)
Pneumonia['Count of Stay'] = Pneumonia.groupby(['病歷號碼'])['住院號'].rank(method="first", ascending=True)


# Length og Stay base on Exam Day
Pneumonia_exam = pd.merge(Pneumonia, ExamDay[['病歷號碼', '住院號', '採檢日_Earliest']].drop_duplicates(), how='left' , on=['病歷號碼', '住院號'])
Pneumonia_exam['Length of Stay (Exam)'] = (Pneumonia_exam.出院日 - Pneumonia_exam.採檢日_Earliest).dt.days


TableOne_JIM = pd.merge(Pneumonia_exam, TableJim[['Patient_Code', '性別', '死亡日期', 'ICU', 'Death', 'AGE', 'Over_18']].drop_duplicates(), how='left' , on='Patient_Code')
# TableOne_MRSA.to_excel(r'C:\Users\User\VSCode\CDSS\output\MRSA_TableOne.xlsx', index=False)

In [ ]:
TableOne_JIM_WhithoutLeave = TableOne_JIM.dropna(subset=['出院日'])
# TableOne_JIM.to_excel(r"C:\Users\User\VSCode\CDSS\output\Pneumonia_TableOne.xlsx", index=False)

2022_06_01    

In [ ]:
Germ = guaguabao[['病歷號碼', '住院號', '菌名']].drop_duplicates()

Germ_condition = [
    (Germ.菌名.isin(['Staphylococcus aureus(MRSA)', 'Staphylococcus aureus'])),
    (Germ.菌名.isin(['Klebsiella pneumoniae'])),
    (Germ.菌名.isin(['Enterococcus faecium(VRE)', 'Enterococcus faecium'])),
    (Germ.菌名.isin(['Acinetobacter baumannii', 'Acinetobacter baumannii/calcoaceticus complex'])),
    (Germ.菌名.isin(['Pseudomonas aeruginosa'])),
    (Germ.菌名.isin(['Escherichia coli'])),
    (Germ.菌名.isin(['Enterococcus faecalis', 'Enterococcus faecalis(VRE)'])),
    (Germ.菌名.isin(['Morganella morganii']))
]

Germ_value = ['SA', 'KP', 'EF', 'AB', 'PA', 'ECL', 'EFC', 'MM'
    
]

Germ['Group_Of_Bacteria'] = np.select(Germ_condition, Germ_value)

Germ = Germ[['病歷號碼', 'Group_Of_Bacteria']].drop_duplicates()
Germ['Count_Of_Bacteria'] = Germ.groupby('病歷號碼')['Group_Of_Bacteria'].transform('count')

Germ = Germ[['病歷號碼', 'Count_Of_Bacteria']].drop_duplicates()


TableOne_JIM_Germ = pd.merge(TableOne_JIM_WhithoutLeave, Germ, how='left', on='病歷號碼')

# TableOne_JIM_Germ.to_excel(r"C:\Users\User\VSCode\CDSS\output\Pneumonia_TableOne.xlsx", index=False)

In [ ]:
TableOne_JIM_Germ[TableOne_JIM_Germ.Over_18==1].count()

2022_06_15

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

In [17]:
Med = pd.read_excel(r'C:\Users\User\VSCode\CDSS\source\資訊室\admdata_1110606.xlsx', dtype={'住院號碼':'str'})
LookUp_Tb = pd.read_excel(r'C:\Users\User\VSCode\CDSS\source\抗生素藥品對照表.xlsx', dtype={'病歷號碼':'str'})

In [7]:
print(Med.columns)
print()
print(LookUp_Tb.columns)

Index(['住院號碼', '醫令代碼', '醫令學名', '使用起日', '使用迄日', '頻率', '每次量', '途徑', '劑型', '天數'], dtype='object')

Index(['藥品分類', 'ATC code ', '健保碼', '院方藥品碼', '抗生素名稱', ' 給藥途徑', 'DDD', '單位'], dtype='object')


In [18]:
LookUp_Tb = LookUp_Tb.dropna(axis=0, subset=['院方藥品碼'])
LookUp_Tb.rename(columns={"院方藥品碼": "醫令代碼"}, inplace = True)
LookUp_Tb = LookUp_Tb[['藥品分類', '醫令代碼', 'DDD', '單位']]

conditions = [
                (LookUp_Tb['醫令代碼'] == 'ITEICOP'), (LookUp_Tb['醫令代碼'] == 'IVANCOM3'),
                (LookUp_Tb['醫令代碼'] == 'IDAPTOM'), (LookUp_Tb['醫令代碼'] == 'ILINEZO'),
                (LookUp_Tb['醫令代碼'] == 'ICEFTA6'),(LookUp_Tb['醫令代碼'] == 'ITIGECY')
             ]
values = ( 1 , 1 , 1 , 1 , 1 ,1)

LookUp_Tb['AntiMRSA'] = np.select(conditions , values)



In [43]:
LookUp_Tb['MRSA_drugs'] = np.where(LookUp_Tb.groupby('藥品分類')['AntiMRSA'].transform('sum')>=1, 1, 0)

In [44]:
Med[['使用起日_date', '使用起日_time']] = Med['使用起日'].str.split(' ', expand=True)
Med[['使用迄日_date', '使用迄日_time']] = Med['使用迄日'].str.split(' ', expand=True)

Med[['使用起日_time_h', '使用起日_time_m']] = Med['使用起日_time'].str.split(':', expand=True)
Med[['使用迄日_time_h', '使用迄日_time_m']] = Med['使用迄日_time'].str.split(':', expand=True)

Med['使用起日_time_m'] = np.where(Med['使用起日_time_m'].astype('int')>59, '30', Med['使用起日_time_m'])
Med['使用迄日_time_m'] = np.where(Med['使用迄日_time_m'].astype('int')>59, '30', Med['使用迄日_time_m'])

Med['使用起日_date'] = Med['使用起日_date'].astype('int') + 19110000
Med['使用迄日_date'] = Med['使用迄日_date'].astype('int') + 19110000

Med['使用起日'] = pd.to_datetime(Med['使用起日_date'].astype('str') + ' '+ Med['使用起日_time_h'] + ':' + Med['使用起日_time_m'] + ':00')
Med['使用迄日'] = pd.to_datetime(Med['使用迄日_date'].astype('str') + ' '+ Med['使用迄日_time_h'] + ':' + Med['使用迄日_time_m'] + ':00')



AttributeError: Can only use .str accessor with string values!

In [ ]:
Med.columns

Index(['住院號碼', '醫令代碼', '醫令學名', '使用起日', '使用迄日', '頻率', '每次量', '途徑', '劑型', '天數',
       '使用起日_date', '使用起日_time', '使用迄日_date', '使用迄日_time', '使用起日_time_h',
       '使用起日_time_m', '使用迄日_time_h', '使用迄日_time_m'],
      dtype='object')

In [45]:
Med = Med[['住院號碼', '醫令代碼', '醫令學名', '使用起日', '使用迄日', '頻率', '每次量', '途徑', '劑型', '天數']].drop_duplicates().reset_index(drop=True)




Med_ClearDay = pd.merge(Med, LookUp_Tb, how='left', on='醫令代碼').drop_duplicates().dropna(axis=0, subset=['藥品分類']).reset_index(drop=True)



In [46]:

Med_ClearDay['使用起日_min'] = Med_ClearDay.groupby(['住院號碼', '藥品分類'])['使用起日'].transform('min')
Med_ClearDay['使用迄日_Max'] = Med_ClearDay.groupby(['住院號碼', '藥品分類'])['使用迄日'].transform('max')

Med_ClearDay = Med_ClearDay[['住院號碼', '藥品分類', '使用起日', '使用迄日', '使用起日_min', '使用迄日_Max', 'MRSA_drugs']].drop_duplicates().sort_values(['住院號碼', '藥品分類', '使用起日', '使用迄日'])
Med_ClearDay['使用起日_Next'] = Med_ClearDay.groupby(['住院號碼', '藥品分類'])['使用起日'].shift(-1)

Med_ClearDay['Med_Interval'] = (Med_ClearDay['使用起日_Next'] - Med_ClearDay['使用迄日']).dt.days

Med_ClearDay['Med_Interval_Bool'] = np.where((Med_ClearDay['Med_Interval']<-1) | (Med_ClearDay['Med_Interval']>1), 1, 0)

In [47]:
aabb = Med_ClearDay[Med_ClearDay['Med_Interval_Bool'] == 1]

aabb['Med_Interval_fix'] = (aabb['使用迄日'] - aabb['使用起日'] ).dt.days


Med_ClearDay = pd.merge(Med_ClearDay, aabb[['住院號碼', '藥品分類', '使用起日', '使用迄日','Med_Interval', 'Med_Interval_Bool', 'Med_Interval_fix']], how='left', on=['住院號碼', '藥品分類', '使用起日', '使用迄日','Med_Interval', 'Med_Interval_Bool'])

# 增加判別欄
conditions = [Med_ClearDay.Med_Interval.isna(), Med_ClearDay.Med_Interval_Bool == 1]
values = [1, 1]
Med_ClearDay['Med_Test'] = np.select(conditions, values)

C:\Users\User\AppData\Local\Temp\ipykernel_22520\1238919967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aabb['Med_Interval_fix'] = (aabb['使用迄日'] - aabb['使用起日'] ).dt.days


In [48]:
Med_ClearDay['group'] = 0
a = Med_ClearDay['Med_Test'].tolist()
b = []
c = Med_ClearDay['group'].tolist()
j = 0
k = 0

for i in range(len(a)):
    if a[i] == 1:
        b.append(list(map(lambda x:x+k, c[j:i+1])))
        j = i+1
        k +=1

# 2D-list to 1D-list
import itertools
flat_list = list(itertools.chain(*b))

# 存入group
Med_ClearDay['group'] = flat_list

del a, b, c, i, j, k 

In [49]:
Med_ClearDay['start_day'] = Med_ClearDay.groupby(['住院號碼','group'])['使用起日'].transform('min')
Med_ClearDay['end_day'] = Med_ClearDay.groupby(['住院號碼','group'])['使用迄日'].transform('max')
Med_ClearDay['start_day'] = pd.to_datetime(Med_ClearDay['start_day'], format='%Y-%m%d').dt.date
Med_ClearDay['end_day'] = pd.to_datetime(Med_ClearDay['end_day'], format='%Y-%m%d').dt.date
Med_ClearDay['day_diff'] = (Med_ClearDay['end_day'] - Med_ClearDay['start_day']).dt.days

In [51]:
result = Med_ClearDay[['住院號碼', '藥品分類', 'group', 'start_day', 'end_day', 'day_diff', 'MRSA_drugs']].drop_duplicates()
result['day_diff'] = result['day_diff'].add(1)
# result.to_excel(r'C:\Users\User\VSCode\CDSS\source\Clean\抗生素用藥天數.xlsx', index=False)